# MASTER Thesis 

## Nordic Countries

In [130]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings 
import datetime as dt
warnings.filterwarnings('ignore')
from scipy import stats

# load excel file with all deals. 
data = pd.read_excel('european_PBO_SBO_Majority.xlsx')

# Load excel with all IPO deals
IPO_data = pd.read_excel('European_IPO.xlsx')

# Filter Buyout deals and Secondary buyout deals
data_PBO = data[data['DEAL TYPES'] == 'Buyout']
data_SBO = data[data['DEAL TYPES'] == 'Secondary Buyout']

# get the DEAL DATE to datetime format
data_PBO['DEAL DATE'] = pd.to_datetime(data_PBO['DEAL DATE'])
data_SBO['DEAL DATE'] = pd.to_datetime(data_SBO['DEAL DATE'])
IPO_data['DEAL DATE'] = pd.to_datetime(IPO_data['DEAL DATE'])

# count number of deals for buyout and secondary buyout
# print(f'There are {len(data_PBO)} Buyout deals and {len(data_SBO)} Secondary Buyout deals in the dataset.')

# Divide the dataframe into nordic, western europe and UK, southern europe and eastern europe regions 

# Nordic countries: Denmark, Finland, Iceland, Norway, Sweden
nordic_countries = ['Denmark', 'Finland', 'Iceland', 'Norway', 'Sweden']
PBO_nordic = data_PBO[data_PBO['TARGET COMPANY COUNTRY'].isin(nordic_countries)]
SBO_nordic = data_SBO[data_SBO['TARGET COMPANY COUNTRY'].isin(nordic_countries)]


# Count number of buyout and Secondary buyout in the nordic region:
print(f'There are {len(PBO_nordic)} Buyout deals in the Nordic region.')
print(f'There are {len(SBO_nordic)} Secondary Buyout deals in the Nordic region.') 

# Get number of Deals per year for each region
def get_deals_per_year(df):
    df['Year'] = df['DEAL DATE'].dt.year
    deals_per_year = df.groupby('Year').size().reset_index(name='Number of Deals')
    return deals_per_year


print(f'Number of Deals per Year in Nordic Region for Buyout Deals: \n{get_deals_per_year(PBO_nordic)}')
print(f'Number of Deals per Year in Nordic Region for Secondary Buyout Deals: \n{get_deals_per_year(SBO_nordic)}')
print(f'Number of IPO Deals per Year in Europe: \n{get_deals_per_year(IPO_data)}')


# Make a dataframe for IPO deals in the nordic region
IPO_nordic = IPO_data[IPO_data['TARGET COMPANY COUNTRY'].isin(nordic_countries)]


print(f'Nordic IPO Deals: \n{IPO_nordic}')
print(f'Number of IPO Deals in Nordic Region: {len(IPO_nordic)}')



# Look up companies that have an IPO after a Buyout or Secondary Buyout exit
def find_IPO_after_exit(buyout_df, ipo_df):
    buyout_df = buyout_df[['TARGET COMPANY', 'DEAL DATE']] # find buyout companies and deal date
    ipo_df = ipo_df[['TARGET COMPANY', 'DEAL DATE']] # find ipo companies and deal date
    merged_df = pd.merge(buyout_df, ipo_df, on='TARGET COMPANY', suffixes=('_buyout', '_ipo')) # merge dataframes on TARGET COMPANY
    merged_df = merged_df[merged_df['DEAL DATE_ipo'] > merged_df['DEAL DATE_buyout']] # filter for IPOs that happened after Buyout or Secondary Buyout
    return merged_df[['TARGET COMPANY', 'DEAL DATE_buyout', 'DEAL DATE_ipo']].drop_duplicates() # drop duplicates


# Find companies in Nordic region that had an IPO after a Buyout or Secondary Buyout exit with format %Y-%m-%d
IPO_after_PBO_nordic = find_IPO_after_exit(PBO_nordic, IPO_nordic)
IPO_after_SBO_nordic = find_IPO_after_exit(SBO_nordic, IPO_nordic)

# print the IPO after Buyout and Secondary Buyout exits in Nordic region
print(f'Companies in Nordic region that had an IPO after a Buyout exit: \n{IPO_after_PBO_nordic}')
print(f'Companies in Nordic region that had an IPO after a Secondary Buyout exit: \n{IPO_after_SBO_nordic}')
    

# for each IPO in the nordic region calculate the time difference in days between the Buyout and the IPO EXIT
def calculate_time_to_IPO(ipo_after_exit_df):
    ipo_after_exit_df['Time to IPO (days)'] = (ipo_after_exit_df['DEAL DATE_ipo'] - ipo_after_exit_df['DEAL DATE_buyout']).dt.days
    return ipo_after_exit_df[['TARGET COMPANY', 'DEAL DATE_buyout', 'DEAL DATE_ipo', 'Time to IPO (days)']]

# print(f'Time IPO happens after buyout is {calculate_time_to_IPO(IPO_after_PBO_nordic)} days for Nordic region Buyout deals.')



There are 669 Buyout deals in the Nordic region.
There are 200 Secondary Buyout deals in the Nordic region.
Number of Deals per Year in Nordic Region for Buyout Deals: 
    Year  Number of Deals
0   2007               34
1   2008               38
2   2009               19
3   2010               26
4   2011               34
5   2012               32
6   2013               22
7   2014               42
8   2015               41
9   2016               54
10  2017               54
11  2018               57
12  2019               43
13  2020               47
14  2021               65
15  2022               61
Number of Deals per Year in Nordic Region for Secondary Buyout Deals: 
    Year  Number of Deals
0   2007                5
1   2008                3
2   2009                5
3   2010                7
4   2011                5
5   2012                9
6   2013               10
7   2014               13
8   2015               16
9   2016               17
10  2017               11
11  20

## Import peer group 

In [ ]:
# Read Peers data
peers_data = pd.read_excel('Mappe2.xlsx')

# Get the nordic buyouts


# export the buyout nordic data to excel and IPO deal data 
PBO_nordic.to_excel('PBO_nordic.xlsx', index=False)
IPO_after_PBO_nordic.to_excel('IPO_after_PBO_nordic.xlsx', index=False)

PBO_nordic

peers_data

# calculate YoY revenue growth for each company in peers_data where REVENUE T-3 is the first year and T + 3 is the last year
peers_data['YoY Revenue Growth T-3 to T-2'] = (peers_data['REVENUE T-2'] / peers_data['REVENUE T-3']) 
peers_data['YoY Revenue Growth T-2 to T-1'] = (peers_data['REVENUE T-1'] / peers_data['REVENUE T-2']) 
peers_data['YoY Revenue Growth T-1 to T'] = (peers_data['REVENUE T'] / peers_data['REVENUE T-1']) 
peers_data['YoY Revenue Growth T to T+1'] = (peers_data['REVENUE T+1'] / peers_data['REVENUE T']) 
peers_data['YoY Revenue Growth T+1 to T+2'] = (peers_data['REVENUE T+2']  / peers_data['REVENUE T+1']) 
peers_data['YoY Revenue Growth T+2 to T+3'] = (peers_data['REVENUE T+3']  / peers_data['REVENUE T+2']) 
# calculate average YoY revenue growth for each company in peers_data
peers_data['Average YoY Revenue Growth'] = peers_data[['YoY Revenue Growth T-3 to T-2', 'YoY Revenue Growth T-2 to T-1', 'YoY Revenue Growth T-1 to T', 'YoY Revenue Growth T to T+1', 'YoY Revenue Growth T+1 to T+2', 'YoY Revenue Growth T+2 to T+3']].mean(axis=1)
print(f'Peers data with YoY revenue growth and average YoY revenue growth: \n{peers_data}')



# Calculate the same for Nordic Buyouts
PBO_nordic['YoY Revenue Growth T-3 to T-2'] = (PBO_nordic['REVENUE T-2'] / PBO_nordic['REVENUE T-3'])
PBO_nordic['YoY Revenue Growth T-2 to T-1'] = (PBO_nordic['REVENUE T-1'] / PBO_nordic['REVENUE T-2']) 
PBO_nordic['YoY Revenue Growth T-1 to T'] = (PBO_nordic['REVENUE T'] / PBO_nordic['REVENUE T-1']) 
PBO_nordic['YoY Revenue Growth T to T+1'] = (PBO_nordic['REVENUE T+1'] / PBO_nordic['REVENUE T'])
PBO_nordic['YoY Revenue Growth T+1 to T+2'] = (PBO_nordic['REVENUE T+2']  / PBO_nordic['REVENUE T+1'])
PBO_nordic['YoY Revenue Growth T+2 to T+3'] = (PBO_nordic['REVENUE T+3']  / PBO_nordic['REVENUE T+2'])


# Divide them into two groups where T-3 to T is before the Buyout and T to T+3 is after the Buyout
PBO_nordic_before = PBO_nordic[['TARGET COMPANY', 'YoY Revenue Growth T-3 to T-2', 'YoY Revenue Growth T-2 to T-1', 'YoY Revenue Growth T-1 to T']]
PBO_nordic_after = PBO_nordic[['TARGET COMPANY', 'YoY Revenue Growth T to T+1', 'YoY Revenue Growth T+1 to T+2', 'YoY Revenue Growth T+2 to T+3']]
# calculate average YoY revenue growth for each company in PBO_nordic_before and PBO_nordic_after
PBO_nordic_before['Average YoY Revenue Growth Before Buyout'] = PBO_nordic_before[['YoY Revenue Growth T-3 to T-2', 'YoY Revenue Growth T-2 to T-1', 'YoY Revenue Growth T-1 to T']].mean(axis=1)
PBO_nordic_after['Average YoY Revenue Growth After Buyout'] = PBO_nordic_after[['YoY Revenue Growth T to T+1', 'YoY Revenue Growth T+1 to T+2', 'YoY Revenue Growth T+2 to T+3']].mean(axis=1)

# divide Control group (peers data) into two groups where T-3 to T is before the Buyout and T to T+3 is after the Buyout
peers_data_before = peers_data[['Company', 'YoY Revenue Growth T-3 to T-2', 'YoY Revenue Growth T-2 to T-1', 'YoY Revenue Growth T-1 to T']]
peers_data_after = peers_data[['Company', 'YoY Revenue Growth T to T+1', 'YoY Revenue Growth T+1 to T+2', 'YoY Revenue Growth T+2 to T+3']]

# calculate average YoY revenue growth for each company in peers_data_before and peers_data_after
peers_data_before['Average YoY Revenue Growth'] = peers_data_before[['YoY Revenue Growth T-3 to T-2', 'YoY Revenue Growth T-2 to T-1', 'YoY Revenue Growth T-1 to T']].mean(axis=1)
peers_data_after['Average YoY Revenue Growth'] = peers_data_after[['YoY Revenue Growth T to T+1', 'YoY Revenue Growth T+1 to T+2', 'YoY Revenue Growth T+2 to T+3']].mean(axis=1)


# difference in average YoY revenue growth before and after the Buyout for PBO_nordic and peer data
before = PBO_nordic_before['Average YoY Revenue Growth Before Buyout'].mean() - peers_data_before['Average YoY Revenue Growth'].mean()
after = PBO_nordic_after['Average YoY Revenue Growth After Buyout'].mean() - peers_data_after['Average YoY Revenue Growth'].mean()

print(f'Difference in average YoY revenue growth before the Buyout for PBO_nordic and peer data: {before}')
print(f'Difference in average YoY revenue growth after the Buyout for PBO_nordic and peer data: {after}')

# find if there is a statisitcal significance between the Two groups using T-test on the average YoY revenue growth before and after the Buyout and compare it to the peers data 
t_stat_before, p_value_before = stats.ttest_ind(PBO_nordic_before['Average YoY Revenue Growth Before Buyout'].dropna(), peers_data_before['Average YoY Revenue Growth'].dropna(), equal_var=False)
t_stat_after, p_value_after = stats.ttest_ind(PBO_nordic_after['Average YoY Revenue Growth After Buyout'].dropna(), peers_data_after['Average YoY Revenue Growth'].dropna(), equal_var=False)
print(f'T-test results for Average YoY Revenue Growth Before Buyout: t-statistic = {t_stat_before}, p-value = {p_value_before}')
print(f'T-test results for Average YoY Revenue Growth After Buyout: t-statistic = {t_stat_after}, p-value = {p_value_after}')


# now make a difference in difference analysis to see if there is a significant difference in the average YoY revenue growth before and after the Buyout for PBO_nordic and peer data
# create a new dataframe for the difference in difference analysis
diff_data_after = pd.DataFrame()
diff_data_after['Company'] = pd.concat([PBO_nordic_after['TARGET COMPANY'], peers_data_after['Company']])
diff_data_after['Average YoY Revenue Growth'] = pd.concat([PBO_nordic_after['Average YoY Revenue Growth After Buyout'], peers_data_after['Average YoY Revenue Growth']])
diff_data_after['Group'] = ['Treated'] * len(PBO_nordic_after) + ['Control'] * len(peers_data_after)
diff_data_after['Time'] = ['Before'] * len(diff_data_after)
diff_data_after['Interaction'] = np.where((diff_data_after['Group'] == 'Treated') & (diff_data_after['Time'] == 'Before'), 1, 0)
# run the difference in difference analysis using OLS regression
model_before = smf.ols('Q("Average YoY Revenue Growth") ~ Group + Time + Interaction', data=diff_data_after).fit()
print(model_before.summary())

# # now account for the age of the company at the time of Buyout 
# PBO_nordic['COMPANY AGE AT BUYOUT'] = PBO_nordic['DEAL DATE'].dt.year - PBO_nordic['YEAR ESTABLISHED']
# diff_data_before = diff_data_before.merge(PBO_nordic[['TARGET COMPANY', 'COMPANY AGE AT BUYOUT']], left_on='Company', right_on='TARGET COMPANY', how='left')
# diff_data_before['COMPANY AGE AT BUYOUT'] = diff_data_before['COMPANY AGE AT BUYOUT'].fillna(diff_data_before['COMPANY AGE AT BUYOUT'].mean())
# model_before_age = smf.ols('Q("Average YoY Revenue Growth") ~ Group + Time + Interaction + Q("COMPANY AGE AT BUYOUT")', data=diff_data_before).fit()
 





## other countries

In [ ]:
# Southern Europe countries: Portugal, Spain, Italy, Greece
southern_europe_countries = ['Portugal', 'Spain', 'Italy', 'Greece']
PBO_southern = data_PBO[data_PBO['TARGET COMPANY COUNTRY'].isin(southern_europe_countries)]
SBO_southern = data_SBO[data_SBO['TARGET COMPANY COUNTRY'].isin(southern_europe_countries)]

# Western Europe countries: France, Germany, Netherlands, Belgium, Austria, Switzerland
western_europe_countries = ['France', 'Germany', 'Netherlands', 'Belgium', 'Austria', 'Switzerland', 'Luxembourg', 'Monaco', 'Liechtenstein']
PBO_western = data_PBO[data_PBO['TARGET COMPANY COUNTRY'].isin(western_europe_countries)]
SBO_western = data_SBO[data_SBO['TARGET COMPANY COUNTRY'].isin(western_europe_countries)]


UK_countries = ['UK', 'Ireland']
# UK
PBO_UK = data_PBO[data_PBO['TARGET COMPANY COUNTRY'].isin(UK_countries)]
SBO_UK = data_SBO[data_SBO['TARGET COMPANY COUNTRY'].isin(UK_countries)]
# Print number of deals in each region
print(f'Nordic: {len(PBO_nordic)} Buyout deals and {len(SBO_nordic)} Secondary Buyout deals.')
print(f'Southern Europe: {len(PBO_southern)} Buyout deals and {len(SBO_southern)} Secondary Buyout deals.')
print(f'Western Europe: {len(PBO_western)} Buyout deals and {len(SBO_western)} Secondary Buyout deals.')
print(f'UK: {len(PBO_UK)} Buyout deals and {len(SBO_UK)} Secondary Buyout deals.')

# Eastern Europe countries: Poland, Czech Republic, Hungary, Romania, Bulgaria, Slovakia, Slovenia, Croatia, Serbia, Bosnia and Herzegovina, Montenegro, North Macedonia, Albania
eastern_europe_countries = ['Poland', 'Czech Republic', 'Hungary', 'Romania', 'Bulgaria', 'Slovakia', 'Slovenia', 'Croatia', 'Serbia', 'Bosnia and Herzegovina', 'Montenegro', 'North Macedonia', 'Albania', 'Estonia', 'Latvia', 'Lithuania', 'Ukraine', 'Belarus', 'Moldova', 'Russia', 'Turkey',]
PBO_eastern = data_PBO[data_PBO['TARGET COMPANY COUNTRY'].isin(eastern_europe_countries)]
SBO_eastern = data_SBO[data_SBO['TARGET COMPANY COUNTRY'].isin(eastern_europe_countries)]
print(f'Eastern Europe: {len(PBO_eastern)} Buyout deals and {len(SBO_eastern)} Secondary Buyout deals.')



# Get number of Deals per year for each region
def get_deals_per_year(df):
    df['Year'] = df['DEAL DATE'].dt.year
    deals_per_year = df.groupby('Year').size().reset_index(name='Number of Deals')
    return deals_per_year


print(f'Number of Deals per Year in Southern Europe for Buyout Deals: \n{get_deals_per_year(PBO_southern)}')
print(f'Number of Deals per Year in Southern Europe for Secondary Buyout Deals: \n{get_deals_per_year(SBO_southern)}')
print(f'Number of Deals per Year in Western Europe for Buyout Deals: \n{get_deals_per_year(PBO_western)}')
print(f'Number of Deals per Year in Western Europe for Secondary Buyout Deals: \n{get_deals_per_year(SBO_western)}')
print(f'Number of Deals per Year in UK for Buyout Deals: \n{get_deals_per_year(PBO_UK)}')
print(f'Number of Deals per Year in UK for Secondary Buyout Deals: \n{get_deals_per_year(SBO_UK)}')


IPO_southern = IPO_data[IPO_data['TARGET COMPANY COUNTRY'].isin(southern_europe_countries)]
IPO_western = IPO_data[IPO_data['TARGET COMPANY COUNTRY'].isin(western_europe_countries)]
IPO_UK = IPO_data[IPO_data['TARGET COMPANY COUNTRY'] == 'UK']
IPO_eastern = IPO_data[IPO_data['TARGET COMPANY COUNTRY'].isin(eastern_europe_countries)]


print(f'Number of IPO Deals in Southern Europe: {len(IPO_southern)}')
print(f'Number of IPO Deals in Western Europe: {len(IPO_western)}')
print(f'Number of IPO Deals in UK: {len(IPO_UK)}')
print(f'Number of IPO Deals in Eastern Europe: {len(IPO_eastern)}')


# Find companies in Western Europe that had an IPO after a Buyout or Secondary Buyout exit
IPO_after_PBO_western = find_IPO_after_exit(PBO_western, IPO_western)
IPO_after_SBO_western = find_IPO_after_exit(SBO_western, IPO_western)
print(f'Companies in Western Europe that had an IPO after a Buyout exit: \n{IPO_after_PBO_western}')
print(f'Companies in Western Europe that had an IPO after a Secondary Buyout exit: \n{IPO_after_SBO_western}')

# Find companies in UK that had an IPO after a Buyout or Secondary Buyout exit
IPO_after_PBO_UK = find_IPO_after_exit(PBO_UK, IPO_UK)
IPO_after_SBO_UK = find_IPO_after_exit(SBO_UK, IPO_UK)
print(f'Companies in UK that had an IPO after a Buyout exit: \n{IPO_after_PBO_UK}')
print(f'Companies in UK that had an IPO after a Secondary Buyout exit: \n{IPO_after_SBO_UK}')

# Find companies in Eastern Europe that had an IPO after a Buyout or Secondary Buyout exit
IPO_after_PBO_eastern = find_IPO_after_exit(PBO_eastern, IPO_eastern)
IPO_after_SBO_eastern = find_IPO_after_exit(SBO_eastern, IPO_eastern)
print(f'Companies in Eastern Europe that had an IPO after a Buyout exit: \n{IPO_after_PBO_eastern}')
print(f'Companies in Eastern Europe that had an IPO after a Secondary Buyout exit: \n{IPO_after_SBO_eastern}')

# Find companies in Southern Europe that had an IPO after a Buyout or Secondary Buyout exit
IPO_after_PBO_southern = find_IPO_after_exit(PBO_southern, IPO_southern)
IPO_after_SBO_southern = find_IPO_after_exit(SBO_southern, IPO_southern)
print(f'Companies in Southern Europe that had an IPO after a Buyout exit: \n{IPO_after_PBO_southern}')
print(f'Companies in Southern Europe that had an IPO after a Secondary Buyout exit: \n{IPO_after_SBO_southern}')


Peers data with YoY revenue growth and average YoY revenue growth: 
      Company  IPO                   Industry  YEAR ESTABLISHED  REVENUE T-3  \
0      ABEKAT   NO         Education/Training              2019           29   
1   TISSEMAND   NO          Consumer Products              2020           27   
2   TISSEKONE  YES  Business Support Services              2019           26   
3        LØVE   NO  Business Support Services              2013           19   
4     ELEFANT   NO                  Materials              2000           10   
5      SAHARA   NO  Business Support Services              2017           25   
6      MARCUS  YES               Agribusiness              2019           29   
7      ANDERS   NO                       Food              2010           22   
8       BØLLE   NO       Information Services              2006           27   
9        PRUT   NO   Logistics & Distribution              2009           14   
10       LORT   NO         Financial Services       